In [1]:
# import libraries
from tensorflow import keras
import numpy as np
import os
import time
import datetime
import h5py
import random
from scipy import spatial
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# define the data location and load data
cwd = os.getcwd()
pardir = os.path.dirname(cwd)
data_folder = os.path.join(pardir,"data")
output_folder = os.path.join(pardir, "results", "video_pred_quan_eval_results")
data_path = os.path.join(data_folder,'video_prediction_dataset.hdf5')

print("data_folder:", data_folder)
print("data_path:", data_path)

data_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/data
data_path: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/data/dataset_all/video_prediction_dataset.hdf5


In [3]:
# load testing data
times_test = np.load(os.path.join(data_folder,"dataset_all","times_curr_test.npy"),allow_pickle=True)
print("times_test.shape:", times_test.shape)

with h5py.File(data_path,'r') as f:

    # read in the ground_truth data
    #images_log_test = f['test']['images_log'][...]
    images_pred_test = f['test']['images_pred'][:,::2,:,:,:]

#print("images_log_test.shape:",images_log_test.shape)
print("images_pred_test.shape:",images_pred_test.shape)

times_test.shape: (4467,)
images_pred_test.shape: (4467, 8, 64, 64, 3)


In [4]:
# mask out img background
def mask_background(img): # put all background pixels to 0s
    mask_img = img.copy()
    for i in range(64):
        for j in range(64):
            if (i-30)**2+(j-30)**2>=31**2:
                mask_img[:,:,i,j,:]=0
    return mask_img

In [5]:
# mask out the background
mask_images_pred_test = mask_background(images_pred_test)
mask_images_pred_test = mask_images_pred_test.astype('float32')/255
print(mask_images_pred_test.shape)

(4467, 8, 64, 64, 3)


In [8]:
# calculate VGG cosine similarity
vgg16 = keras.applications.VGG16(weights='imagenet',include_top=True,pooling='max',input_shape=(224,224,3))
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
basemodel=keras.Model(inputs=vgg16.input,outputs=keras.layers.Flatten()(vgg16.get_layer('block4_pool').output))

In [10]:
def get_feature_vector(img):
    
    img1 = img.reshape(-1,224,224,3)
    feature_vector = np.zeros((img1.shape[0],basemodel.output.shape[-1]))
    batch_size = 100
    num_samples = img1.shape[0]
    indices = np.arange(num_samples)
    for i in range(int(num_samples / batch_size) + 1):
    
        start_idx = (i * batch_size) % num_samples
        idxs = indices[start_idx:start_idx + batch_size]
        feature_vector[idxs] = basemodel.predict(img1[idxs])
    
    feature_vector=feature_vector.reshape((img.shape[0],img.shape[1],basemodel.output.shape[-1]))
    return feature_vector

In [16]:
mask_images_pred_test_resized = np.zeros((images_pred_test.shape[0],images_pred_test.shape[1],224,224,3))
for i in range(images_pred_test.shape[0]):
    for j in range(images_pred_test.shape[1]):
        mask_images_pred_test_resized[i,j] = cv2.resize(mask_images_pred_test[i,j],(224,224))
feature_vector_gt_test = (get_feature_vector(mask_images_pred_test_resized*255)).astype('float32')

In [17]:
def calculate_cosine_similarity(vector1, vector2):
    cosine_similarity = 1-spatial.distance.cosine(vector1,vector2)
    return cosine_similarity

### Evaluation of model performance quantitatively

### SkyGPT

In [13]:
# video prediction models for predicted images
def loadData(samp_num, vp_model, vp_model_folder):

    print("vp_model_folder:", vp_model_folder)

    # load predicted images
    predicted_images = np.load(os.path.join(vp_model_folder,samp_num))[:,8:, :, :, :]
    # scale back all the pixel values back to [0,1] with clipping
    predicted_images = np.clip(predicted_images,-0.5,0.5)+0.5
    # scale back the pixel values to [0,255]
    #predicted_images = (predicted_images*255).astype('int')
    # mask out the background
    predicted_images = mask_background(predicted_images.astype('float32'))
    
    predicted_images_resized = np.zeros((predicted_images.shape[0],predicted_images.shape[1],224,224,3))
    for i in range(predicted_images.shape[0]):
        for j in range(predicted_images.shape[1]):
            predicted_images_resized[i,j] = cv2.resize(predicted_images[i,j],(224,224))
    feature_vector_pred_test = (get_feature_vector(predicted_images_resized*255)).astype('float32')
    print("predicted_images.shape:",predicted_images.shape)
    print("feature_vector_pred_test.shape:",feature_vector_pred_test.shape)
    
    return predicted_images, feature_vector_pred_test


In [14]:
def metrics(predicted_images, images_pred_test, feature_vector_gt, feature_vector_pred):    
    mse_per_frame = np.mean(np.sum((predicted_images-images_pred_test)**2,axis=(2,3,4)),axis=0)
    mae_per_frame = np.mean(np.sum(np.abs(predicted_images-images_pred_test),axis=(2,3,4)),axis=0)
    ssim_per_frame = np.zeros((images_pred_test.shape[0],images_pred_test.shape[1]))
    cosine_similarity_per_frame = np.zeros((images_pred_test.shape[0],images_pred_test.shape[1]))
    
    for i in range(images_pred_test.shape[0]):
        for j in range(images_pred_test.shape[1]):
            ssim_per_frame[i,j] = ssim(images_pred_test[i,j], predicted_images[i,j], multichannel=True)
            
            # calcuate vgg cosine similarity
            cosine_similarity_per_frame[i,j] = calculate_cosine_similarity(feature_vector_gt[i,j],feature_vector_pred[i,j])
        
    ssim_per_frame = np.mean(ssim_per_frame, axis=0)
    cosine_similarity_per_frame = np.mean(cosine_similarity_per_frame, axis=0)
    
    np.save(os.path.join(output_folder,'SkyGPT_4x4x4_mse_avg'),mse_per_frame)
    #np.save(os.path.join(output_folder,'SkyGPT_4x4x4_mae_avg'),mae_per_frame)
    #np.save(os.path.join(output_folder,'SkyGPT_4x4x4_ssim_avg'),ssim_per_frame)
    np.save(os.path.join(output_folder,'SkyGPT_4x4x4_cosine_similarity_avg_block4_pool'),cosine_similarity_per_frame)
    
    for j in range(images_pred_test.shape[1]):
        print('-'*50)
        print('evaluation for {0} min'.format(j*2+1))
        print('mse (frame-wise): ', mse_per_frame[j])
        print('mae (frame-wise): ', mae_per_frame[j])
        print('ssim: ', ssim_per_frame[j])
        print('cosine_similarity: ', cosine_similarity_per_frame[j])

    print("*"*50)
    print('overall performance:')
    print('mse (frame-wise): ', np.mean(mse_per_frame))
    print('mae (frame-wise): ', np.mean(mae_per_frame))
    print('ssim: ', np.mean(ssim_per_frame))
    print('cosine_similarity: ', np.mean(cosine_similarity_per_frame))

In [15]:
from tqdm.notebook import tqdm

def metrics_batch(predicted_images, images_pred_test, feature_vector_gt, feature_vector_pred):
    mse_per_frame = np.mean(
        np.min(
            np.sum(
                (predicted_images-images_pred_test[None])**2,
                axis=(3,4,5)
                ),
            axis=0),
        axis=0
    )
    mae_per_frame = np.mean(
        np.min(
            np.sum(
                np.abs(predicted_images-images_pred_test[None]),
                axis=(3,4,5)
            ),
        axis=0),
    axis=0)
    
    cosine_similarity_per_frame = np.zeros((predicted_images.shape[0],images_pred_test.shape[0],images_pred_test.shape[1]))
    
    for i in range(images_pred_test.shape[0]):
        for j in range(images_pred_test.shape[1]):
            for k in range(predicted_images.shape[0]):
                cosine_similarity_per_frame[k,i,j] = calculate_cosine_similarity(feature_vector_gt[i,j],feature_vector_pred[k,i,j])

    cosine_similarity_on_time_step = np.mean(np.max(cosine_similarity_per_frame, axis=0),axis=0)
    
    # take mean across all time stamps
    cosine_similarity_per_frame_mean = np.mean(cosine_similarity_per_frame, axis=2)
    
    # find the best match across all 10 samplings and take the mean of every sample
    cosine_similarity_per_frame_mean = np.mean(np.max(cosine_similarity_per_frame_mean,axis=0),axis=0)
    
    # find the index of the best/worst match across 10 samplings for every sample
    best_match_img_index = np.argmax(cosine_similarity_per_frame_mean,axis=0)
    worst_match_img_index = np.argmin(cosine_similarity_per_frame_mean,axis=0)
    
    np.save(os.path.join(output_folder,'SkyGPT_4x4x4_mse_min'),mse_per_frame)
    np.save(os.path.join(output_folder,'SkyGPT_4x4x4_mae_min'),mae_per_frame)
    np.save(os.path.join(output_folder,'SkyGPT_4x4x4_cosine_similarity_max_block4pool'),cosine_similarity_on_time_step)
    np.save(os.path.join(output_folder,'best_match_img_index_based_on_cosine_similarity_among_10sampling_SkyGPT_4x4x4_block4_pool_features.npy'), best_match_img_index)
    np.save(os.path.join(output_folder,'worst_match_img_index_based_on_cosine_similarity_among_10sampling_SkyGPT_4x4x4_block4_pool_features.npy'), worst_match_img_index)
    
    for j in range(images_pred_test.shape[1]):
        print('-'*50)
        print('evaluation for {0} min'.format(j*2+1))
        print('mse (frame-wise): ', mse_per_frame[j])
        print('mae (frame-wise): ', mae_per_frame[j])
        print('cosine_similarity: ', cosine_similarity_on_time_step[j])
        
    print("*"*50)
    print('overall performance:')
    print('mse (frame-wise): ', np.mean(mse_per_frame))
    print('mae (frame-wise): ', np.mean(mae_per_frame))
    print('cosine_similarity: ', np.mean(cosine_similarity_on_time_step))
    
    return cosine_similarity_per_frame, best_match_img_index, worst_match_img_index

In [16]:
#loop over all samples
vp_model_folder = '/scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/SkyGPT_full_2min_4x4x4'
samp_array = os.listdir(vp_model_folder)
samp_array = np.asarray(samp_array)[['sample' in samp for samp in samp_array]]
print(images_pred_test.shape)

predicted_images_batch = []
feature_vector_pred_test_batch = []

for samp_num in samp_array:
    if len(samp_num.split('.'))==2:
        vp_model = "SkyGPT_2min_4x4x4_" + samp_num

        print('Sample #: ', samp_num)
        predicted_images, feature_vector_pred_test = loadData(samp_num, vp_model, vp_model_folder)
        predicted_images_batch.append(predicted_images.astype('float32'))
        feature_vector_pred_test_batch.append(feature_vector_pred_test)

(4467, 8, 64, 64, 3)
Sample #:  sample_1637223777647.npy
vp_model_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/PhyGPT_full_2min_4x4x4
predicted_images.shape: (4467, 8, 64, 64, 3)
feature_vector_pred_test.shape: (4467, 8, 100352)
Sample #:  sample_1637220835372.npy
vp_model_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/PhyGPT_full_2min_4x4x4
predicted_images.shape: (4467, 8, 64, 64, 3)
feature_vector_pred_test.shape: (4467, 8, 100352)
Sample #:  sample_1637220668023.npy
vp_model_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/PhyGPT_full_2min_4x4x4
predicted_images.shape: (4467, 8, 64, 64, 3)
feature_vector_pred_test.shape: (4467, 8, 100352)
Sample #:  sample_1637223937443.npy
vp_model_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/PhyGPT_full_2min_4x4x4
predicted_images.shape: (4467, 8, 64, 64, 3)
feature_v

In [22]:
cosine_similarity, closest_img_index, worst_match_img_index = metrics_batch(np.stack(predicted_images_batch), mask_images_pred_test, feature_vector_gt_test, np.stack(feature_vector_pred_test_batch))

--------------------------------------------------
evaluation for 1 min
mse (frame-wise):  29.457476
mae (frame-wise):  328.25455
ssim:  0.8797790675196445
cosine_similarity:  0.762326207034455
--------------------------------------------------
evaluation for 3 min
mse (frame-wise):  42.16947
mae (frame-wise):  395.97812
ssim:  0.8491638853461537
cosine_similarity:  0.7445709356825064
--------------------------------------------------
evaluation for 5 min
mse (frame-wise):  52.70917
mae (frame-wise):  443.44574
ssim:  0.8291056422091154
cosine_similarity:  0.7312985944126126
--------------------------------------------------
evaluation for 7 min
mse (frame-wise):  60.507515
mae (frame-wise):  479.59885
ssim:  0.8165373222712371
cosine_similarity:  0.7211482022762405
--------------------------------------------------
evaluation for 9 min
mse (frame-wise):  66.075165
mae (frame-wise):  500.84088
ssim:  0.807376802888151
cosine_similarity:  0.712467585730558
------------------------------

In [23]:
metrics(np.stack(predicted_images_batch).mean(0), mask_images_pred_test,feature_vector_gt_test,np.stack(feature_vector_pred_test_batch).mean(0))

--------------------------------------------------
evaluation for 1 min
mse (frame-wise):  27.776663
mae (frame-wise):  327.86334
ssim:  0.8868968508135886
cosine_similarity:  0.7940797558801137
--------------------------------------------------
evaluation for 3 min
mse (frame-wise):  41.279846
mae (frame-wise):  404.74927
ssim:  0.8567342504290391
cosine_similarity:  0.78322625836724
--------------------------------------------------
evaluation for 5 min
mse (frame-wise):  54.971214
mae (frame-wise):  463.6721
ssim:  0.8347877488897169
cosine_similarity:  0.7728684017635283
--------------------------------------------------
evaluation for 7 min
mse (frame-wise):  65.65722
mae (frame-wise):  511.33798
ssim:  0.8205330465599966
cosine_similarity:  0.7635067370958928
--------------------------------------------------
evaluation for 9 min
mse (frame-wise):  71.619
mae (frame-wise):  531.87335
ssim:  0.8132021323322705
cosine_similarity:  0.7592612222814442
--------------------------------

In [24]:
predicted_images_batch_stacked = np.stack(predicted_images_batch)
feature_vector_pred_test_batch_stacked = np.stack(feature_vector_pred_test_batch)

In [21]:
np.save(os.path.join(output_folder,'predicted_images_10sampling_stacked_mask_background_SkyGPT_4x4x4.npy'),predicted_images_batch_stacked)

In [6]:
predicted_images_batch_stacked = np.load(os.path.join(output_folder,'predicted_images_10sampling_stacked_mask_background_SkyGPT_4x4x4.npy'))
best_match_img_index = np.load(os.path.join(output_folder,'best_match_img_index_based_on_cosine_similarity_among_10sampling_SkyGPT_4x4x4_block4_pool_features.npy'))